### Importing libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
import pickle
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
from scikeras.wrappers import KerasClassifier


### Dataset import

In [2]:
df=pd.read_csv("Churn_Modelling.csv")
df.sample(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9062,9063,15753110,McKay,720,Spain,Male,64,3,45752.78,2,1,0,79623.28,1
1118,1119,15759381,Johnson,617,Spain,Male,61,7,91070.43,1,1,1,101839.77,0
3229,3230,15589715,Fulks,584,France,Female,66,5,0.00,1,1,0,49553.38,1
9903,9904,15778959,Brookes,606,France,Female,36,10,0.00,2,0,1,155641.46,0
1577,1578,15576714,Manna,687,Spain,Female,21,8,0.00,2,1,1,154767.34,0


## Data Preprocessing

In [3]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [4]:
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1)
df.sample(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
8119,704,France,Male,31,5,132084.66,3,1,1,54474.48,1
867,636,France,Female,48,1,170833.46,1,1,0,110510.28,1
9829,584,France,Male,38,1,115341.55,1,0,1,173632.92,0
4277,775,Germany,Male,51,2,123783.25,1,1,1,134901.57,0
1869,652,Germany,Male,33,7,128135.99,1,1,0,158437.73,0


### OneHotEncoder

In [5]:
ohe=OneHotEncoder()
geog_ohe=ohe.fit_transform(df[["Geography"]]).toarray()
geog_ohe

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [6]:
ohe.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
geog_df=pd.DataFrame(geog_ohe,columns=ohe.get_feature_names_out())
geog_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [8]:
df=pd.concat([df,geog_df],axis=1)
df.sample(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
6898,575,Spain,Male,41,2,100062.39,1,0,0,126307.25,0,0.0,0.0,1.0
1204,630,France,Female,40,7,0.00,2,1,1,34453.17,0,1.0,0.0,0.0
4265,709,France,Male,32,4,147307.91,1,0,1,40861.55,0,1.0,0.0,0.0
5780,604,Germany,Female,42,10,166031.45,1,1,0,98293.14,0,0.0,1.0,0.0
2694,628,Germany,Male,29,3,113146.98,2,0,1,124749.08,0,0.0,1.0,0.0


In [9]:
df=df.drop(["Geography"],axis=1)
df.sample(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
3607,682,Male,37,5,0.00,2,0,1,112554.68,0,1.0,0.0,0.0
1870,624,Male,33,6,0.00,2,0,0,76551.70,0,0.0,0.0,1.0
4416,471,Male,42,3,164951.56,1,1,0,190531.77,0,1.0,0.0,0.0
5934,765,Male,34,9,91835.16,1,0,0,138280.17,0,1.0,0.0,0.0
5150,515,Female,37,0,196853.62,1,1,1,132770.11,0,1.0,0.0,0.0


In [10]:
df.sample(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
2238,835,Male,28,2,163569.61,2,1,1,154559.28,0,1.0,0.0,0.0
6653,608,Male,23,8,197715.93,2,1,1,116124.28,0,0.0,1.0,0.0
3004,596,Male,47,5,140187.10,2,1,1,174311.30,0,0.0,1.0,0.0
5768,674,Male,36,2,154525.70,1,0,1,27468.72,0,1.0,0.0,0.0
9587,719,Female,76,10,95052.29,1,1,0,176244.87,0,0.0,1.0,0.0


### LabelEncoder

In [11]:
le=LabelEncoder()
df["Gender"]=le.fit_transform(df["Gender"])
df.sample(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
3001,696,0,43,4,0.00,2,1,1,66406.37,0,0.0,0.0,1.0
2165,511,1,33,7,0.00,2,0,1,158313.87,0,1.0,0.0,0.0
7395,721,0,45,7,138523.20,1,0,0,59604.45,1,0.0,1.0,0.0
7276,594,1,35,2,133853.27,1,1,1,65361.66,0,1.0,0.0,0.0
9107,655,1,38,9,0.00,1,0,1,90490.33,0,0.0,0.0,1.0


In [12]:
df["Gender"].value_counts()

Gender
1    5457
0    4543
Name: count, dtype: int64

### Independent and Dependent Features

In [13]:
X=df.drop("Exited",axis=1)
y=df["Exited"]

In [14]:
X.sample(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
7907,683,0,39,2,47685.47,2,1,1,86019.48,0.0,1.0,0.0
8221,443,1,59,4,110939.30,1,1,0,72846.58,0.0,1.0,0.0
4482,653,0,42,1,0.00,2,1,1,5768.32,1.0,0.0,0.0
6138,477,1,47,9,144900.58,1,1,0,61315.37,1.0,0.0,0.0
2595,633,1,29,10,130206.28,1,1,0,184654.87,1.0,0.0,0.0


In [15]:
y.sample(5)

3985    0
2138    0
4881    0
4888    0
6152    0
Name: Exited, dtype: int64

### Train test split

In [16]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((8000, 12), (8000,), (2000, 12), (2000,))

### Scaling

In [17]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

### Pickling

In [18]:
with open("standard_scaler.pkl", "wb") as file:
    pickle.dump(scaler,file)

with open("ohe_geog.pkl", "wb") as file:
    pickle.dump(ohe,file)

with open("le_gender.pkl", "wb") as file:
    pickle.dump(le,file)
    

### ANN Implementation

In [19]:
model=Sequential([
    Dense(64, activation="relu" , input_shape=(X_train.shape[1],)),
    Dense(32, activation="relu"),
    Dense(1, activation="sigmoid")
]

)

2024-08-19 23:36:13.351055: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-08-19 23:36:13.351075: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-08-19 23:36:13.351080: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-08-19 23:36:13.351107: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-19 23:36:13.351120: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

## Logging

In [22]:
log_dir="logs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir, histogram_freq=1)

In [23]:
earlystopping_callback=EarlyStopping(monitor="val_loss",restore_best_weights=True,patience=10)

### Model fitting

In [24]:
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,earlystopping_callback],verbose=1
)

Epoch 1/100


2024-08-19 23:36:14.475625: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - 5s 7ms/step - loss: 0.4727 - accuracy: 0.7851 - val_loss: 0.4180 - val_accuracy: 0.8130
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4380 - accuracy: 0.8071 - val_loss: 0.4194 - val_accuracy: 0.8130
Epoch 3/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4395 - accuracy: 0.8080 - val_loss: 0.4207 - val_accuracy: 0.8150
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4455 - accuracy: 0.8119 - val_loss: 0.4233 - val_accuracy: 0.8120
Epoch 5/100
250/250 [==============================] - 1s 6ms/step - loss: 0.4454 - accuracy: 0.8080 - val_loss: 0.4233 - val_accuracy: 0.8160
Epoch 6/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4545 - accuracy: 0.8048 - val_loss: 0.4416 - val_accuracy: 0.8205
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4748 - accuracy: 0.8074 - val_loss: 0.4500 - val_accuracy: 0.8100
Epoch 8/100

### Model saving

In [25]:
model.save("Classification_model.h5")

/Users/subhajeetkhawas/Desktop/UdemyGenAIKn/ANN/venv1/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Tensorboard visaulization

In [32]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [33]:
%tensorboard --logdir logs/fit/20240819-205100

Reusing TensorBoard on port 6006 (pid 58921), started 0:18:39 ago. (Use '!kill 58921' to kill it.)

### Hyperparameter Tuning

In [35]:
def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model

In [36]:
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [39]:
param_grid = {
    'neurons': [16, 32],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [40]:
gsv=GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
gsv_result=gsv.fit(X_train,y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


/Users/subhajeetkhawas/Desktop/UdemyGenAIKn/ANN/venv1/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/subhajeetkhawas/Desktop/UdemyGenAIKn/ANN/venv1/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/subhajeetkhawas/Desktop/UdemyGenAIKn/ANN/venv1/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/subhajeetkhawas/Desktop/UdemyGenAIKn/ANN/venv1/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``m

Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50


2024-08-19 23:59:47.661927: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-08-19 23:59:47.664704: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-08-19 23:59:47.666630: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-08-19 23:59:47.672507: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-08-19 23:59:47.695153: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-08-19 23:59:47.712082: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-08-19 23:59:47.721505: I tensorflow/core/grappler/optimizers/cust

167/167 [==============================] - 4s 16ms/step - loss: 0.7306 - accuracy: 0.5948
Epoch 2/50
167/167 [==============================] - 4s 16ms/step - loss: 0.5510 - accuracy: 0.7424
Epoch 2/50
167/167 [==============================] - 4s 17ms/step - loss: 0.5940 - accuracy: 0.6970
Epoch 2/50
167/167 [==============================] - 4s 21ms/step - loss: 0.5379 - accuracy: 0.7401
Epoch 2/50
167/167 [==============================] - 5s 24ms/step - loss: 0.6440 - accuracy: 0.6962
Epoch 2/50
167/167 [==============================] - 3s 20ms/step - loss: 0.4826 - accuracy: 0.8022
Epoch 3/50
167/167 [==============================] - 3s 19ms/step - loss: 0.4580 - accuracy: 0.7996
Epoch 3/50
167/167 [==============================] - 3s 20ms/step - loss: 0.4621 - accuracy: 0.7980
Epoch 3/50
167/167 [==============================] - 3s 17ms/step - loss: 0.4367 - accuracy: 0.8093
Epoch 3/50
167/167 [==============================] - 3s 17ms/step - loss: 0.4449 - accuracy: 0.8056
E

...

167/167 [==============================] - 4s 23ms/step - loss: 0.4351 - accuracy: 0.8033
Epoch 51/100
167/167 [==============================] - 4s 24ms/step - loss: 0.4521 - accuracy: 0.8061
Epoch 3/100
167/167 [==============================] - 3s 20ms/step - loss: 0.4365 - accuracy: 0.8104
Epoch 3/100
Epoch 51/100
  6/167 [>.............................] - ETA: 3s - loss: 0.4316 - accuracy: 0.8125

/Users/subhajeetkhawas/Desktop/UdemyGenAIKn/ANN/venv1/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


167/167 [==============================] - 4s 22ms/step - loss: 0.4673 - accuracy: 0.8014
Epoch 48/100
167/167 [==============================] - 3s 18ms/step - loss: 0.5472 - accuracy: 0.7967
Epoch 48/100
167/167 [==============================] - 3s 16ms/step - loss: 0.4359 - accuracy: 0.8105
Epoch 52/100
167/167 [==============================] - 6s 20ms/step - loss: 0.5107 - accuracy: 0.7671
Epoch 2/100
167/167 [==============================] - 3s 19ms/step - loss: 0.5860 - accuracy: 0.7900
Epoch 49/100
167/167 [==============================] - 6s 23ms/step - loss: 0.5219 - accuracy: 0.7510
Epoch 2/100
167/167 [==============================] - 4s 21ms/step - loss: 0.4395 - accuracy: 0.8106
Epoch 53/100
167/167 [==============================] - 4s 21ms/step - loss: 0.4340 - accuracy: 0.8061
Epoch 5/100
167/167 [==============================] - 4s 21ms/step - loss: 0.4396 - accuracy: 0.8069
Epoch 3/100
167/167 [==============================] - 3s 19ms/step - loss: 0.4358 - accu

/Users/subhajeetkhawas/Desktop/UdemyGenAIKn/ANN/venv1/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/100
250/250 [==============================] - 1s 4ms/step - loss: 0.6029 - accuracy: 0.6909
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4433 - accuracy: 0.8052
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4337 - accuracy: 0.8083
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4330 - accuracy: 0.8114
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4328 - accuracy: 0.8110
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4334 - accuracy: 0.8100
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4332 - accuracy: 0.8102
Epoch 8/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4327 - accuracy: 0.8087
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4330 - accuracy: 0.8085
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4336 - accura

In [41]:
gsv_result.best_score_

0.8126260098957997

In [43]:
gsv_result.best_params_

{'epochs': 100, 'layers': 1, 'neurons': 16}

### New model creation

In [44]:
model=Sequential([
    Dense(16, activation="relu" , input_shape=(X_train.shape[1],)),
    Dense(1, activation="sigmoid")
]
)

In [45]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 16)                208       
                                                                 
 dense_6 (Dense)             (None, 1)                 17        
                                                                 
Total params: 225 (900.00 Byte)
Trainable params: 225 (900.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [46]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [47]:
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,earlystopping_callback],verbose=1
)

Epoch 1/100
250/250 [==============================] - 1s 5ms/step - loss: 0.5585 - accuracy: 0.7218 - val_loss: 0.4492 - val_accuracy: 0.8110
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4436 - accuracy: 0.8075 - val_loss: 0.4202 - val_accuracy: 0.8170
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4352 - accuracy: 0.8119 - val_loss: 0.4169 - val_accuracy: 0.8105
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4338 - accuracy: 0.8110 - val_loss: 0.4177 - val_accuracy: 0.8175
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4340 - accuracy: 0.8095 - val_loss: 0.4170 - val_accuracy: 0.8160
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4339 - accuracy: 0.8130 - val_loss: 0.4152 - val_accuracy: 0.8155
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4337 - accuracy: 0.8120 - val_loss: 0.4160 - val_accuracy: 0.8165

### Saving new model

In [48]:
model.save("Classification_model.h5")

/Users/subhajeetkhawas/Desktop/UdemyGenAIKn/ANN/venv1/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
